In [1]:
import numpy as np
import pandas as pd
import re
import random

In [2]:
random.seed(777)

In [3]:
# Load the original dataset and delete null values of price, sellers, stock and category
dataset = pd.read_csv(filepath_or_buffer='originalDataset.csv')
dataset = dataset[dataset['features.value'].notna()]
dataset = dataset[dataset['name'].notna()]
dataset = dataset[dataset['brand'].notna()]

In [4]:
dataset = dataset[dataset['features.key'] == 'Ingredients']

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8990 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  8990 non-null   object 
 1   asins               5367 non-null   object 
 2   brand               8990 non-null   object 
 3   categories          8990 non-null   object 
 4   dateAdded           8990 non-null   object 
 5   dateUpdated         8990 non-null   object 
 6   ean                 6242 non-null   object 
 7   features.key        8990 non-null   object 
 8   features.value      8990 non-null   object 
 9   manufacturer        6319 non-null   object 
 10  manufacturerNumber  5992 non-null   object 
 11  name                8990 non-null   object 
 12  sizes               247 non-null    object 
 13  upc                 7966 non-null   object 
 14  weight              5670 non-null   object 
 15  Unnamed: 15         0 non-null      float64
dtypes: flo

In [6]:
# dataset['features.key'].value_counts()
# dataset.groupby(['name','features.key'])['features.key'].count()
# dataset.query('select * from dataset');

In [7]:
# dataset.tail()

In [8]:
attributes = ['name','brand','categories','features.key','features.value']
dataset[attributes]

,name,brand,categories,features.key,features.value
0,Simon Fischer Fruit Bttr Prune Lekvar,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery",Ingredients,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin."
1,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,McCormick,"Grocery & Gourmet Food,Food,Grocery",Ingredients,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,..."
2,Jolly Time Popcorn,Jolly Time,"Grocery & Gourmet Food,Grocery",Ingredients,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ..."
3,Ziyad Tahini Sesame Sauce,Ziyad,"Grocery & Gourmet Food,grocery",Ingredients,Mechanically hulled seasame seeds.Allergy Info...
4,Fla-Vor-Ice Plus Giant Pops,Fla-Vor-Ice,"Grocery & Gourmet Food,grocery",Ingredients,FALSE
...,...,...,...,...,...
9995,Wright's Liquid Smoke - 3.5 Oz,Wright's,"Barbecue,Grocery & Gourmet Food,Sauces, Gravie...",Ingredients,"water,natural hickory smoke flavor."
9996,Eden Foods Organic Pocket Snacks - Tamari Almo...,Eden Foods,"Food,Snacks, Cookies & Chips,Nuts & Trail Mixe...",Ingredients,"Organic Dry Roasted Almonds,Organic Tamari Soy..."
9997,"Happy Belly Fancy Whole Cashews, 44 Ounce",Happy Belly,"Cashews,Grocery & Gourmet Food,Cooking & Bakin...",Ingredients,"Cashews, Vegetable Oil (May Contain One Or Mor..."
9998,QuakerÂ® PoppedÂ® Cheddar Cheese Rice Crisps 6...,Quaker,"Food,Snacks, Cookies & Chips,Rice Cakes,Chips,...",Ingredients,"Whole Grain Brown Rice Flour,Degerminated Mill..."


In [9]:
def removePatterns(str, patterns, rep = ''):
    for pattern in patterns:
        str = re.sub(pattern, rep, str)
    return str

def getIngredients(key, value):
    # Remove patterns: :zcx., :(zcx), (asd), points, asteriscs, -, +, !, #, >, numbers, percentages
    removals = ['\:.*?\.', '\:\(.*?\)', '\(.*?\)', '\.', '\*', '-', '\+', '!', '#', '>', '[0-9]+[ ]*', '%[ ]*',
                '\(', '\)', '\[', '\]']
    value = removePatterns(value, removals)
    # To lower
    value = value.lower()
    # Split
    ingredients = re.split('and | and|/| or |;|,|&', value)
    # Format all ingredients
    for i in range(len(ingredients)):
        ingredients[i] = ingredients[i].strip()
    #Return
    return ingredients
    
def storeTuple(dictionary, idGroup, name, brand, ingredients):
    # EACH INGREDIENT GENERATES A NEW TUPLE
    for ingredient in ingredients:
        if ingredient != '' and len(ingredient) < 100:
            tuple = {'idGroup':idGroup, 'name':name, 'brand':brand, 'component':ingredient}
            dictionary.append(tuple)

strRemovals = ["'", '"', '[^a-zA-Z0-9 (),;]+']
dictionary = []
idGroup = 0
for tuple in dataset.iterrows():
    name = removePatterns(tuple[1]['name'], strRemovals)
    brand = removePatterns(tuple[1]['brand'], strRemovals)
    ingredients = getIngredients(tuple[1]['features.key'], tuple[1]['features.value'])
    storeTuple(dictionary, idGroup, name, brand, ingredients)
    idGroup += 1
    
newDataset = pd.DataFrame(dictionary)    

In [10]:
newDataset.tail()

,idGroup,name,brand,component
93409,8989,Alpen All Natural Muesli Original,Weetabix,rolled whole oat flakes
93410,8989,Alpen All Natural Muesli Original,Weetabix,brown sugar
93411,8989,Alpen All Natural Muesli Original,Weetabix,sliced almonds
93412,8989,Alpen All Natural Muesli Original,Weetabix,malt extract
93413,8989,Alpen All Natural Muesli Original,Weetabix,roasted hazelnuts


In [11]:
# ========================================= STORING DATASET ==============================================
# ======== MY RELATIONAL CONDITION FORMAT
def getHeader(row, sizes, columns=[]):
    header = ""
    if(len(columns) == 0): header = "id(float,1);"
    currSize = 0
    currType = ""
    i = 0
    for title in row:
        # Verify the column is in the list
        if len(columns) == 0 or title in columns:
            if isinstance(row[title], str):
                currType = "string"
                currSize = sizes[i] + 1
            else:
                currType = "float"
                currSize = 1
            # Add element to header
            header += title + "(" + currType + "," + str(currSize) + ");"
        # Increment
        i += 1
    # printing the header
    return header

def getFormat(value):
    if isinstance(value, str):
        return '"' + str(value) + '"; '
    else:
        return str(value) + "; "
    

def saveData(dictionary, sizes):
    f = open("../T1.data", "w")
    f.write(getHeader(dictionary[0], sizes)+"\n")
    for i in range(len(dictionary)):
        f.write(str(i)+"; ")
        for value in dictionary[i].items():
            f.write(getFormat(value[1]))
        f.write("\n")
    f.close()
    
def saveGroups(dictionary):
    f = open("../TG.data", "w")
    for idRow in range(len(dictionary)):
        f.write(str(idRow) + "\t" + str(dictionary[idRow]["idGroup"]) + "\n")
    f.close()

In [12]:
def createTests(dictionary, sizes, columns):
    random.seed(20)
    # For each case from 2 to 100
    for iTest in range(2,101):
        # Open the folder of current test case
        f = open("../TESTS/TEST "+str(iTest)+ "/T2.data", "w")
        # Store the header
        f.write(getHeader(dictionary[0], sizes, columns) + "\n")
        # Variable number of requirements
        sizeT2 = int(random.random()*9 + 2)
        # Generate all the requirements
        for iReq in range(sizeT2):
            iTuple = int(random.random()*len(dictionary))
            # Iterating columns
            for value in dictionary[iTuple].items():
                if value[0] in columns:
                    f.write(getFormat(value[1]))
            # end tuple with end line
            f.write("\n")
        # Close current test file
        f.close()
    


In [13]:
# ========== SQL DATABASE FORMAT    

def getSQLCreateTable(row, tableName, sizes):
    header = "DROP TABLE IF EXISTS " + tableName + ";\n"
    header += "CREATE TABLE " + tableName + "(\n"
    header += "\tid INT PRIMARY KEY \n"
    currSize = 0
    currType = ""
    i = 0
    for title in row:
        # Separation between attributes
        header += "\t, "
        # Find the type of the attribute
        if isinstance(row[title], str):
            currType = "VARCHAR"
            currSize = sizes[i] + 1
            # Add element to header
            header += title + " " + currType + "(" + str(currSize) + ")\n"
        else:
            currType = "FLOAT"
            header += title + " " + currType + "\n"
        
        # Increment
        i += 1
    # closing the sentence
    header += ");\n"
    # printing the instruction
    return header

def getSQLFormat(value):
    if isinstance(value, str):
        return ", \"" + str(value) + "\""
    else:
        return ', ' + str(value)

def saveSQLData(dictionary, tableName, sizes, pgSize = 10000):
    f = open("AllergiesDB.sql", "w")
    f.write(getSQLCreateTable(dictionary[0], tableName, sizes)+"\n")
    # Each tuple in the dataset
    for idRow in range(len(dictionary)):
        # Separation between tuples and pages
        if idRow % pgSize == 0:
            if idRow > 0: f.write(";\n")
            f.write("INSERT INTO " + tableName + " VALUES\n")
        else:
            f.write(", ")
        # Start writing the tuple
        f.write("(" + str(idRow)) # id
        # Each attribute of the tuple
        for value in dictionary[idRow].items():
            f.write(getSQLFormat(value[1]))
        f.write(")\n")
    # Closing last page
    if len(dictionary) % pgSize > 0:
        f.write(";\n")
    # Closing file
    f.close()
    



In [14]:
# Max length of strings in the new dataset
measurer = np.vectorize(len)
sizes = measurer(newDataset.values.astype(str)).max(axis=0)
# print(sizes)
saveData(dictionary, sizes)
saveGroups(dictionary)
saveSQLData(dictionary, "T1", sizes)

In [15]:
# Create tests 2 - 100
createTests(dictionary, sizes, ['component'])